# Article Labeling & Lexical Fields Finding

In [1]:
import pandas as pd
import numpy as np
import sys
import re

In [ ]:
import json

data = []
with open('./data/20200420_20200714_business_articles.json') as f:
    for line in f:
        data.append(json.loads(line))


In [ ]:
print ("data type",type (data))
print ("json",type (data[0]))
print ("keys",data[0].keys())
print ("length", len(data))

In [ ]:
# extracting full_text of each article:

articles = list()
min_length = 1000000
min_article_size = 2000
for article in data:
    plain_text = article.get('full-text')
    if (plain_text and "Article `download()` failed" != plain_text[:27] and "Please enable cookies" != plain_text[:21] and len(plain_text)>min_article_size):
        articles.append(plain_text)
        # Check for small articles
        #if len(plain_text)<min_length:
        #    min_length = len(plain_text)
        #    print("*****************")
        #    print (plain_text)
        #    print("*****************")

In [ ]:
# 358192 removing "Article `download()` failed" 
# 340987 removing "Article `download()` failed" and "Please enable cookies"
# 215039 removing "Article `download()` failed" and "Please enable cookies" and size<min_article_size = 2000
print ("len(articles)",len(articles) )
print ("min_length",min_length)

#for i in range (3001, 3002, 1):
#    print ("article", i)
#    print ("article", i, articles[i])


In [ ]:
# Cleaning full_text of articles
# Remove every non-letter character
clean_articles = list()

for article in articles[:100]:
    clean_text = article.lower()
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_text = re.sub("[^a-z]", ' ', clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_articles.append(clean_text)

In [ ]:
clean_articles[0]

In [ ]:
# Python program to generate word vectors using Word2Vec 
  
# importing all necessary modules 
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
nltk.download('punkt')
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

import collections

In [ ]:
#pip install -U scikit-learn scipy matplotlib

In [ ]:
# Testing
#tokenised = word_tokenize(clean_articles[0])
#tokenised
#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(clean_articles)
#X.toarray()

In [66]:
# Reads ‘alice.txt’ file 
#sample = open("C:\\Users\\Admin\\Desktop\\alice.txt", "r") 
#s = sample.read() 
  
# Replaces escape character with space 
#f = s.replace("\n", " ") 
  
data = [] 
  
# iterate through each article in the file 
for i in clean_articles: 
    temp = [] 
    # tokenize the article into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

# Create CBOW model 
model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5) 
  
# Print results 
print("Cosine similarity between 'australia' " + 
               "and 'melbourne' - CBOW : ", 
    model1.similarity('melbourne', 'australia')) 

print(model1.wv.most_similar('melbourne'))
    

# Create Skip Gram model 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, 
                                             window = 5, sg = 4) 
  
# Print results 
print("Cosine similarity between 'australia' " +
          "and 'melbourne' - Skip Gram : ", 
    model2.similarity('melbourne', 'australia')) 
print(model2.wv.most_similar('melbourne'))


Cosine similarity between 'australia' and 'melbourne' - CBOW :  0.9946784
[('reopen', 0.995359480381012), ('forced', 0.995207667350769), ('average', 0.9952065944671631), ('chinese', 0.9951968789100647), ('winning', 0.995181679725647), ('demonstration', 0.9951574802398682), ('hit', 0.9951567649841309), ('association', 0.995130717754364), ('judge', 0.9951156377792358), ('lane', 0.9951006770133972)]
Cosine similarity between 'australia' and 'melbourne' - Skip Gram :  0.99351615
[('commitments', 0.9994041323661804), ('adam', 0.9993471503257751), ('delivery', 0.9992997646331787), ('eyes', 0.9992934465408325), ('itself', 0.9992783069610596), ('smash', 0.9992754459381104), ('shes', 0.9992724657058716), ('researches', 0.9992630481719971), ('sensitive', 0.9992622137069702), ('devices', 0.999258279800415)]


In [74]:
len(data)
# Show frequencies
#print("Original List : ",data)
data_flat = []
for line in data:
    for word in line:
        data_flat.append(word)


ctr = collections.Counter(data_flat)
print("Frequency of the elements in the List : ",ctr)
ctr

Frequency of the elements in the List :  Counter({'the': 4045, 'to': 2168, 'of': 1853, 'in': 1817, 'a': 1702, 'and': 1702, 'on': 800, 'for': 659, 'that': 636, 'with': 549, 'said': 523, 'was': 455, 'is': 447, 'as': 423, 'at': 413, 's': 406, 'it': 404, 'by': 398, 'from': 373, 'have': 333, 'he': 323, 'police': 322, 'be': 288, 'has': 283, 'not': 283, 'are': 277, 'were': 271, 'his': 270, 'an': 245, 'who': 238, 'new': 235, 'its': 228, 'their': 222, 'black': 222, 'after': 221, 'but': 220, 'will': 219, 'this': 212, 'we': 212, 'had': 208, 'i': 205, 'people': 195, 'her': 179, 'u': 175, 'year': 174, 'about': 173, 'been': 170, 'she': 161, 'more': 161, 'they': 160, 'reuters': 152, 'would': 149, 'up': 148, 'which': 147, 'or': 138, 'coronavirus': 137, 'over': 133, 'other': 131, 'while': 128, 'some': 124, 'city': 120, 'one': 117, 'also': 117, 'no': 117, 'minneapolis': 116, 'when': 114, 'out': 114, 'first': 113, 'three': 109, 'than': 109, 'all': 108, 'into': 108, 'during': 105, 'two': 104, 'york': 103,

Counter({'eliminated': 6,
         'masterchef': 6,
         'contestant': 3,
         'harry': 5,
         'foster': 2,
         'has': 283,
         'hit': 29,
         'back': 78,
         'at': 413,
         'unfair': 3,
         'criticism': 4,
         'against': 92,
         'judge': 12,
         'melissa': 8,
         'leong': 4,
         'the': 4045,
         'show': 24,
         's': 406,
         'first': 113,
         'female': 5,
         'faced': 10,
         'a': 1702,
         'barrage': 2,
         'of': 1853,
         'trolling': 2,
         'with': 549,
         'haters': 2,
         'taking': 23,
         'aim': 3,
         'everything': 10,
         'from': 373,
         'her': 179,
         'behaviour': 6,
         'on': 800,
         'set': 53,
         'to': 2168,
         'fashion': 3,
         'sense': 6,
         'despite': 32,
         'being': 84,
         'tuesday': 66,
         'night': 55,
         'episode': 2,
         'had': 208,
         'nothing': 4

In [71]:
data_flat

array([list(['eliminated', 'masterchef', 'contestant', 'harry', 'foster', 'has', 'hit', 'back', 'at', 'unfair', 'criticism', 'against', 'judge', 'melissa', 'leong', 'the', 'show', 's', 'first', 'female', 'judge', 'has', 'faced', 'a', 'barrage', 'of', 'trolling', 'with', 'haters', 'taking', 'aim', 'at', 'everything', 'from', 'her', 'behaviour', 'on', 'set', 'to', 'her', 'fashion', 'sense', 'despite', 'being', 'eliminated', 'on', 'tuesday', 'night', 's', 'episode', 'harry', 'had', 'nothing', 'but', 'good', 'things', 'to', 'say', 'about', 'the', 'melbourne', 'based', 'food', 'writer', 'this', 'could', 'not', 'be', 'further', 'from', 'the', 'truth', 'eliminated', 'masterchef', 'australia', 'contestant', 'harry', 'foster', 'pictured', 'has', 'hit', 'back', 'at', 'unfair', 'criticism', 'against', 'judge', 'melissa', 'leong', 'she', 's', 'a', 'queen', 'i', 'love', 'her', 'harry', 'told', 'huffpost', 'australia', 'she', 'is', 'energetic', 'passionate', 'and', 'really', 'just', 'vibrant', 'when